In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

Using TensorFlow backend.


In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):     
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    if ds.BodyPartExamined != 'CHEST' or ds.Modality != 'DX':
        return None
    
    if ds.PatientPosition != 'PA' and ds.PatientPosition != 'AP':
        return None
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_scale_factor,img_size): 
    proc_img = resize(img, img_size)
    proc_img *= img_scale_factor
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = keras.models.model_from_json(model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img)
    
    if prob > thresh:
        prediction = "Pneumonia"
    else:
        prediction = "No Pneumonia"
    
    return prediction 

In [5]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "f1_my_model.best.hdf5".format('xray_class')#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_scale_factor = 1 / 255. # loads the mean image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.52983433 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        print('Invalid image!')
        continue
        
    img_proc = preprocess_image(img,img_scale_factor,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
No Pneumonia
Load file test3.dcm ...
No Pneumonia
Load file test4.dcm ...
Invalid image!
Load file test5.dcm ...
Invalid image!
Load file test6.dcm ...
Invalid image!
